# Scheduler-oriented night summary for Rubin Observatory for {{ params.day_obs }} of baseline simulation {{ params.sim_version }}

In [ ]:
# This cell is only for setting parameter defaults

if True:
    day_obs = "2025-11-01"
    new_visit_origin = "3.4"
    previous_visit_origin = ""
else:
    day_obs = "2024-06-26"
    new_visit_origin = "lsstcomcamsim"
    previous_visit_origin = ""

## Introduction

This report is a summary of a night of observing designed to understand scheduler behavior during the night, identify scheduler problems, and otherwise monitor scheduler performance.
It present, it creates reports only from opsim simulations.

In [ ]:
# Uncomment for development
# %load_ext lab_black

In [ ]:
known_instruments = ['lsstcomcamsim']

In [ ]:
# Validate the input
import re

assert re.match(r"^\d\d\d\d-\d\d-\d\d$", day_obs) is not None
assert new_visit_origin in known_instruments or re.match(r"^\d+\.\d+$", new_visit_origin) is not None
assert previous_visit_origin == "" or re.match(r"^\d+\.\d+$", previous_visit_origin) is not None

In [ ]:
import datetime
import logging
import os
import sqlite3
import sys
import warnings

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display
from lsst.resources import ResourcePath

In [ ]:
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/user/neilsen/data/rubin_sim_data"

In [ ]:
sys.path.insert(
    0, "/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages"
)

In [ ]:
devel_versions = True
if devel_versions:
    if False:
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-09-13/rubin_scheduler")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-09-13/rubin_sim")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-09-16/schedview")
    else:
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
        sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")

In [ ]:
import schedview.collect.consdb
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.astro
import schedview.compute.visits
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.collect.visits
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

In [ ]:
# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*dubious year.*",
)

In [ ]:
# Don't complain about working with daytime MJDs either.
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.skybrightness_pre.sky_model_pre",
    category=UserWarning,
    message="Requested MJD between sunrise and sunset, returning closest maps",
)

In [ ]:
# Quiet unimportant chatter from healpy.
healpy_logger = logging.getLogger("healpy")
healpy_logger.setLevel(logging.WARNING)

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
use_matplotlib = True

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

In [ ]:
visits = schedview.collect.visits.read_night_visits(day_obs, new_visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS)

In [ ]:
day_obs

In [ ]:
DayObs.from_date(day_obs).date.isoformat()

## Night narrative

Human written summaries and narratives describing the night and any problems or engineering tests that occurred during it will be important for understandnig what events in the night arose from the scheduler, and what came from other causes.

Tools for recording such text and providing access to it are still under development (see the #rubinobs-nightlog channel in the LSSTC slack), and content not included here.
But, if it could be reported now, here is where it would be.

## Sun and Moon

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)
night_events

Modified Julian Date (MJD) is in units of days (UTC).

Local Sidereal Time (LST) is in units of degrees.

In [ ]:
# Refine the mjd of the middle of the night, unlikely to be important.
observatory.mjd = night_events.loc["night_middle", "MJD"]

In [ ]:
body_positions = schedview.compute.astro.compute_sun_moon_positions(observatory)
body_positions

RA, dec, alt, and az are all in units of degrees.

## Conditions and statistics

### Numbers of exposures, and gaps between them

In [ ]:
if len(visits):
    overhead_summary = schedview.compute.visits.compute_overhead_summary(
        visits,
        night_events.loc["sun_n12_setting", "MJD"],
        night_events.loc["sun_n12_rising", "MJD"],
    )
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    display(HTML(summary_table))
else:
    print("No visits")

## Visit map

In [ ]:
if len(visits):
    visits = schedview.compute.visits.add_coords_tuple(visits)
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs.date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

## Survey Progress

### Map depth accumulated so far

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
if previous_visit_origin != "":
    previous_visits = schedview.collect.read_opsim(
            previous_visit_origin,
            constraint=f"observationStartMjd < {night_events.loc['sunset', 'MJD']}",
        )
else:
    # Empty data from with the same structure as the current visits
    previous_visits = visits.iloc[0:0]

In [ ]:
if len(previous_visits):
    fig = schedview.plot.survey.create_metric_visit_map_grid(
        maf.SumMetric(col="t_eff", metric_name="Total Teff"),
        previous_visits,
        visits,
        observatory,
        nside=32,
        use_matplotlib=use_matplotlib
    )

Each subplot represents the progress in the survey in a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

Annotations are similar to those of the "Visit Map" above:

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The blue backdrop represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) at the start of the night.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

### Map the most recent visit so far

In [ ]:
if len(previous_visits):
    fig = schedview.plot.survey.create_metric_visit_map_grid(
        maf.AgeMetric(night_events.loc['sunset', 'MJD']),
        previous_visits,
        visits,
        observatory,
        nside=32,
        use_matplotlib=use_matplotlib,
        vmin=0,
        vmax=10,
        cmap=colorcet.cm.blues_r
    )

Each subplot represents the time since the most recent visits to each point in the sky, by filter, presented in a Lambert Azimuthal Equal Area Projection, centered at the south celestial pole.

Parts of the sky visited most recently are dark blue, growing fainter and grayer for areas visited less recently. Regions not visited in the last 10 days are not colored.

Other features and annotations are the same as those of the depth map, above.

## DDF Cadence

In [ ]:
visits.columns

In [ ]:
ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visits, previous_visit_origin)

In [ ]:
if len(ddf_visits):
    nightly_ddf = schedview.compute.visits.accum_teff_by_night(ddf_visits)
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs_mjd - time_window_duration, day_obs_mjd
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")

The y-axis (height of the vertical bars) represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) accumulated over all exposures on the field for the night, colored by filter.

## Histogram of gaps between exposures

In [ ]:
if len(visits):
    p1 = schedview.plot.create_overhead_histogram(visits)
    p2 = schedview.plot.plot_overhead_vs_slew_distance(visits)
    overhead_plots = bokeh.layouts.row([p1, p2])
    bokeh.io.show(overhead_plots)
else:
    print("No visits")

"Overhead" is the time window (in seconds) between the end of the previous exposure and the end of the current exposure, minus the exposure time.

## Long gaps between exposures

In [ ]:
if len(visits):
    num_gaps = 10
    visits["previous_filter"] = visits["filter"].shift(1)
    long_gap_visits = (
        visits.sort_values("overhead", ascending=False)
        .query("overhead>30")
        .loc[:, ["start_date", "overhead", "slewDistance", "filter", "previous_filter"]]
        .sort_values("start_date")
    )
    display(long_gap_visits)
else:
    print("No visits")

These are the longest gaps between exposures that occur on the night, ordered by duration of the gaps.

Slew time and previous and current filters are noted, because filter changes are the greatest expected contributions to overhead.

## Value timelines

Select a value to plot vs. time using the dropdown on the left.

In [ ]:
if len(visits):
    fig = schedview.plot.plot_visit_param_vs_time(visits, 'skyBrightness', show_column_selector=True)
    bokeh.io.show(fig)
else:
    print("No visits")

## Table of exposures

In [ ]:
# The bokeh table widget and the Rubin RSP notebooks do not work together.
# If running on the Rubin RSP notebook aspect, set on_rsp=True.
# Otherwise, leave it as False
on_rsp = False

In [ ]:
if len(visits):
    for time_column in 'start_date', 'observationStartDatetime64':
        if time_column in visits:
            break
    displayed_columns = [
        time_column,
        "fieldRA",
        "fieldDec",
        "filter",
        "visitExposureTime",
        "numExposures",
        "t_eff",
        "skyBrightness",
        "seeingFwhmEff",
        "cloud",
        "note",
    ]
    if on_rsp:
        with pd.option_context('display.max_rows', 2000):
            display(visits.loc[:,displayed_columns])
    else:
        schedview.plot.create_visit_table(visits, visible_column_names=displayed_columns, width=1024)
else:
    print("No visits")